<a href="https://colab.research.google.com/github/cpvivek/Facial-Emotion-Recognition/blob/main/Capstone_DL%26MLE_Facial_Emotion_Recognition_Vivek_CP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facial Emotion Detection
In this project, we are aiming to develop a CNN netwrok to detect 7 different human emotions in real time over a live web-cam feed. We'll also be deploying a web app to run this model.

Importing necessary libraries including tensorflow, keras, sklearn and opencv

In [1]:
# import necessary files
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import keras
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential 
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten

Importing dataset sourced from kaggle

In [3]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


reading the csv file.

In [32]:
df = pd.read_csv('/content/drive/MyDrive/Alma Better Pro/Alma Better Pro Program/Module 4: Machine Learning/Data Sets/icml_face_data.csv')
for i in range(len(df[' pixels'])):
  df[' pixels'][i] = np.fromstring(df[' pixels'][i],sep=' ',dtype='float32')
  df[' pixels'][i] = np.asarray(df[' pixels'][i]).reshape(48,48,1)
train_data = df[df[' Usage']=='Training']
val_data = df[df[' Usage']=='PublicTest']
test_data = df[df[' Usage']=='PrivateTest']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Let's look into our dataset

In [5]:
df.columns

Index(['emotion', ' Usage', ' pixels'], dtype='object')

So the dataset is not all that complicated.

We've got 'emotion' field indicating different emotions.

I believe since this is a dataset used in kaggle competitions, they've went ahead and done the training, validation and test segregation, labeled by 'Usage' column

And finally in the 'pixels' field, we have the pixelated form of the image, flattened into an 1 dimension array. 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1    Usage   35887 non-null  object
 2    pixels  35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


The pixel field is stored as a string at the moment. We'll have to convert that into a 48x48x1 array with float values.

In [5]:
#using lambda function to achieve this conversion
df[' pixels']=df[' pixels'].apply(lambda x: np.fromstring(x, sep=' ',dtype='float32')) #converting string to float separated by ' ' 
df[' pixels']=df[' pixels'].apply(lambda x:np.asarray(x.reshape(48,48,1))) #reshaping to 48x48x1

Let's look into emotions field.

In [6]:
df['emotion'].unique()

array([0, 2, 4, 6, 3, 5, 1])

The dataset contains 7 different emotions labeled using number from 0 to 6.

Let's segregate our training, validation and test sets right away. 

Public Test would be put into validation and Private test into test set. 

In [38]:
df[' Usage'].value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name:  Usage, dtype: int64

In [33]:
#defininng train, val and test data.
training_data=df[df[' Usage']=='Training']
validation_data=df[df[' Usage']=='PublicTest']
testing_data=df[df[' Usage']=='PrivateTest']

In [27]:
training_data['emotion'].value_counts()

3    7215
6    4965
4    4830
2    4097
0    3995
5    3171
1     436
Name: emotion, dtype: int64

We can see that there is considerable amount of imbalance in the dataset wrt to emotions. This will create bias in the model and result in misclassifying. 

We can treat this issue by data augmentation.

Using keras for data augmentation.

In [34]:
# Data Augmentation
from keras import layers
data_augmentation=keras.Sequential(
    [
     layers.RandomFlip('horizontal'), #introduce some random flips on horizontal axis
     layers.RandomRotation(0.015), #introduce some rotation by 0.015 degree
     layers.RandomZoom(0.15,0.15) #introduce some random zoom
    ]
)

Applying Data Augmentation on training samples so that each emotion sample has equal number of samples

In [35]:
aug_df=pd.DataFrame({}) #creating a neww dataframe that will contain the augmented images
emotion=[] #list for emotion labels
aug_pixels=[] #list for augemnted image pixels
sample_length=7500 #specifying desired number of samples for each emotions. arbitrary choice

for i in range(7): #since 7 emotions
  data=training_data[training_data['emotion']==i] #selecting the image to be augmented
  j=sample_length - len(data) #calculating number of samples required to reach 7500
  for k in range(j):
    ind= k%len(data) #setting index
    augmented_image=data_augmentation(data[' pixels'][data.index[ind]]) #augmenting images
    aug_pixels.append(augmented_image) #appending augmented image to 
    emotion.append(i)#appending emotion label

aug_df[' pixels'] = aug_pixels #column containing augmented pixels
aug_df['emotion']= emotion#corresponding emotion label

In [36]:
#count of augmented images that will be added for each emotions
aug_df['emotion'].value_counts()

1    7064
5    4329
0    3505
2    3403
4    2670
6    2535
3     285
Name: emotion, dtype: int64

In [37]:
# concat train data and augmented df
training_data=pd.concat([training_data,aug_df],axis=0)
training_data['emotion'].value_counts()

0    7500
2    7500
4    7500
6    7500
3    7500
5    7500
1    7500
Name: emotion, dtype: int64

Since augmented data and training data index wouldn't matchup, we need set up proper index for the new training dataframe

In [38]:
ind=[ind for ind in range(len(training_data))]
training_data.index=ind
training_data.tail()

,emotion,Usage,pixels
52495,6,NaN,"(((tf.Tensor(185.42809, shape=(), dtype=float3..."
52496,6,NaN,"(((tf.Tensor(87.50913, shape=(), dtype=float32..."
52497,6,NaN,"(((tf.Tensor(146.42558, shape=(), dtype=float3..."
52498,6,NaN,"(((tf.Tensor(194.2313, shape=(), dtype=float32..."
52499,6,NaN,"(((tf.Tensor(171.77687, shape=(), dtype=float3..."


So we have 52500 datapoints now. Ignore the NaN values in usage. We know the usage is training

Preprocessing X_train using keras

In [42]:
x_train=[]
for i in range(len(training_data[' pixels'])):
  x_train.append(training_data[' pixels'][i])
x_train=np.asarray(x_train) #since we need arrays as input to CNN 
x_train=x_train.reshape(len(x_train),48,48,1)
y_train=np.array(training_data['emotion'])
y_train=y_train.astype(int)
y_train=np_utils.to_categorical(y_train,7)
# shape of training data
x_train.shape, y_train.shape

((52500, 48, 48, 1), (52500, 7))

Preprocessing X_val using keras

In [43]:
x_val = []
for i in validation_data[' pixels']:
  x_val.append(i)
x_val=np.asarray(x_val)
x_val= x_val.reshape(len(x_val),48,48,1)
y_val=np.array(validation_data['emotion'])
y_val=y_val.astype(int)
y_val=np_utils.to_categorical(y_val,7)



x_val.shape,y_val.shape

((3589, 48, 48, 1), (3589, 7))

Preprocessing x_test using keras

In [44]:
x_test = []

for i in testing_data[' pixels']:
  x_test.append(i)
x_test=np.array(x_test)
x_test=x_test.reshape(len(x_test),48,48,1)
y_test=np.array(test_data['emotion'])
y_test=y_test.astype(int)
y_test=np_utils.to_categorical(y_test,7)
# code here

# then check shape
x_test.shape,y_test.shape

((3589, 48, 48, 1), (3589, 7))

Building models usign keras layers and keras sequential model

In [56]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 46, 46, 64)        640       
                                                                 
 conv2d_9 (Conv2D)           (None, 46, 46, 64)        36928     
                                                                 
 batch_normalization_7 (Batc  (None, 46, 46, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 23, 23, 64)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 23, 23, 128)      

Applying optimization technique

In [57]:
opt = tf.keras.optimizers.Adam(lr=1.0e-3)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) 

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [58]:
tf.config.run_functions_eagerly(True)

In [ ]:
model.fit(x_train,y_train,epochs=10,batch_size=64,validation_data=(x_val,y_val))